In [1]:
import pandas as pd
import time, datetime, itertools
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from urllib.request import Request, urlopen

In [2]:
# We require a tool from google called chromedriver, it can be downloaded at https://chromedriver.chromium.org/

path_to_chromedriver = 'C:\\Users\\Usuario\\Desktop\\chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [3]:
#Select the initial URL you want to manipulate

url = ("https://prv.tutor.com/nGEN/login.aspx?ProgramGUID=B611858B-4D02-4AFE-8053-D082BBC1C58E&ReturnUrl"
       "=%2fnGEN%2fTools%2fScheduleManager_v2%2fsetContactID.aspx%3fProgramGUID%3dB611858B-4D02-4AFE-8053-"
       "D082BBC1C58E%26UserGUID%3d10802325-ae12-496a-82ad-eab04e11aed1")
browser.get(url)
browser.maximize_window()

In [4]:
#The following functions are defined according to the buttons that the webpage displayed, an easy way to find
#the location of a button is by using the x-path. Accordingly, each of the functions listed below correspond
#to a button in the web page. 

def refresh():
    browser.find_element_by_xpath("//*[@id='Form1']/table/tbody/tr[1]/td[1]/strong/a").click()
    
def move_week():
    browser.find_element_by_xpath("//*[@id='Form1']/table/tbody/tr[1]/td[3]/table/tbody/tr/td[3]/img").click()
    
def select_cell( number ):
    xpath = "//*[@id='cell" + str(number) + "']"
    browser.find_element_by_xpath( xpath ).click()

def schedule_selected():
    browser.execute_script("window.alert = function() {};")
    browser.find_element_by_name("butProviderSchedule").click()

def remove_selected():
    browser.execute_script("window.alert = function() {};")
    browser.find_element_by_name("butProviderUnschedule").click()
    
def limit_hour():
    global limit
    limit = int(browser.find_element_by_id( 'lblAvailableHours' ).get_attribute('innerHTML'))
    
def available():
    global a_list, b_list
    a_list , b_list = [] , []
    schedule = browser.find_element_by_xpath("//*[@id='Form1']/table/tbody/tr[2]/td[2]").get_attribute("innerHTML")
    for t in range( len(schedule.split('Available'))-1 ):
        a_list.append(int(schedule.split('Available')[t].split('id="cell')[-1][:-2]))
    for t in range( len(schedule.split('Scheduled'))-1 ):
        b_list.append(int(schedule.split('Scheduled')[t].split('id="cell')[-1][:-2]))
    a_list = sorted(a_list, reverse=True)
    b_list = sorted(b_list, reverse=True)

In [5]:
#Login required by the page

username = browser.find_element_by_id('txtUserName')
password = browser.find_element_by_id('txtPassword')

username.send_keys("correo@gmail.com")
password.send_keys("pass")

browser.find_element_by_name("butSignIn").click()
move_week()

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [7]:
#The following code renders information from the webpage + a code number to characterize each cell

table = OrderedDict()
hours = ['12am'] + [str(t)+'am' for t in range(1,12)] +['12pm']+ [str(t)+'pm' for t in range(1,12)]
for hour in hours:
    h = hours.index(hour)
    table[hour] = {'sun':[7*h] , 'mon':[7*h+1], 'tue':[7*h+2], 'wed':[7*h+3], 'thu':[7*h+4], 'fri':[7*h+5], 'sat':[7*h+6]}
    
available()

for element in itertools.product(*[hours, table['12am'].keys()]):
    l = table[element[0]][element[1]]
    if l[0] in a_list:
        l.append('Available')
    elif l[0] in b_list:
        l.append('Scheduled')
    else:
        l.append(0)

table = pd.DataFrame(table)
table = table.transpose()
table = table[['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']]

def color_yellow(val):
    if val[1] == 'Available':
        color = 'goldenrod' 
    elif val[1]=='Scheduled':
        color = 'green'
    else:
        color = 'black'
    return 'color: %s' % color

table = table.style.applymap(color_yellow)
table 

,sun,mon,tue,wed,thu,fri,sat
12am,"[0, 0]","[1, 'Scheduled']","[2, 0]","[3, 0]","[4, 0]","[5, 0]","[6, 0]"
1am,"[7, 0]","[8, 0]","[9, 0]","[10, 0]","[11, 0]","[12, 0]","[13, 0]"
2am,"[14, 0]","[15, 0]","[16, 0]","[17, 0]","[18, 0]","[19, 0]","[20, 0]"
3am,"[21, 0]","[22, 0]","[23, 0]","[24, 0]","[25, 0]","[26, 0]","[27, 'Available']"
4am,"[28, 0]","[29, 0]","[30, 0]","[31, 0]","[32, 0]","[33, 0]","[34, 0]"
5am,"[35, 0]","[36, 'Scheduled']","[37, 0]","[38, 0]","[39, 0]","[40, 0]","[41, 0]"
6am,"[42, 0]","[43, 0]","[44, 0]","[45, 0]","[46, 0]","[47, 0]","[48, 0]"
7am,"[49, 0]","[50, 0]","[51, 0]","[52, 0]","[53, 0]","[54, 0]","[55, 0]"
8am,"[56, 0]","[57, 0]","[58, 0]","[59, 0]","[60, 0]","[61, 0]","[62, 0]"
9am,"[63, 0]","[64, 0]","[65, 0]","[66, 0]","[67, 0]","[68, 0]","[69, 0]"


In [8]:
#Given that the idea was automate the choosing of the hours, we select the hours we want
#using the previous information (You can modify the code to select all the hours if you want)

set_cells = [83,103,128,132]
len(set_cells)

4

In [12]:
#This portion of the code select the required hours. 

#Part 1------------------------------------------------------------------
refresh()
limit_hour()
while limit == 6:
    refresh()
    limit_hour()

#Part 2------------------------------------------------------------------

select_cell(set_cells[0])

for t in set_cells[1:]:
    select_cell(t)
    schedule_selected()

#Part 3------------------------------------------------------------------
refresh()
available()

while a_list != []:
    
    for k in a_list:
        if 0< k <= 200:
            try:
                select_cell(k)
                schedule_selected()
            except:
                pass

    
    available()
    refresh()
#------------------------------------------------------------------